### Calculate returns across all strike prices for all strategies
#### Option Returns Data Elements
- option_strategy
- entry_date
- call_buy_strike_bucket
- call_sell_strike_bucket
- put_buy_strike_bucket
- put_sell_strike_bucket
- pct_return
- market_direction
- sampling_key

Strike buckets represent to probability that an option at a given strike price will close in the money on expiry date. Using this in place of the actual strike price provides us with a consistent means of comparing returns across time based on the "Moneyness" of an option strike. The probabliity of a strike price being in the money on expiry date will automatically adjust itself based on the implied volatility of the underlying asset; in this case the SPY ETF.
<br/><br/>
Each strategy will have its own output file and only the strike bucket fields relevent to the strategy will be saved

#### The following csv files will be created
- buy_call_returns
- buy_put_returns
- buy_call_spread_returns
- buy_put_spread_returns
- sell_call_spread_returns
- sell_put_spread_returns
- sell_iron_condor_returns



In [1]:
import pandas as pd
import csv
import datetime
from google.cloud import storage
import os

BUCKET_NAME = 'expiry-week-data'

#### Shared Functions

In [2]:
#upload file to Google Cloud Storage 
def upload_blob(source_file_path):
    """Upload a file to Cloud Storage bucket."""
    
    filename = os.path.basename(source_file_path)
    destination_blob_name = "analytics/{}".format(filename)
 
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_path)

    print(
        "File {} uploaded to {}.".format(
            source_file_path, destination_blob_name
        )
    )


In [3]:
def buy_option_return(entry_strike_ask, exit_strike_bid):
    """calculate percentage return for a simple option purchase"""
    pct_return = round((exit_strike_bid - entry_strike_ask) / entry_strike_ask, 3)
    return pct_return

In [4]:
def buy_spread_return(entry_buy_strike_ask, entry_sell_strike_bid, exit_buy_strike_bid, exit_sell_strike_ask):
    """calculate percentage return for a vertical spread"""
    cost = entry_buy_strike_ask - entry_sell_strike_bid
    proceeds = exit_buy_strike_bid - exit_sell_strike_ask
    pct_return = round((proceeds - cost) / cost, 3)
    return pct_return

In [5]:
def sell_spread_return(buy_strike, sell_strike, entry_buy_strike_ask, entry_sell_strike_bid, exit_buy_strike_bid, exit_sell_strike_ask):
    """
    calculate percentage return for a vertical credit spread
     - proceeds will be the initial net credit received less the value of the spread at close out
    """
    net_credit = entry_sell_strike_bid - entry_buy_strike_ask
    margin = abs(buy_strike - sell_strike) - net_credit
    close_out_cost = exit_sell_strike_ask - exit_buy_strike_bid
    pct_return =  round((net_credit - close_out_cost) / margin, 3)
    return pct_return

In [24]:
def iron_condor_return(call_buy_strike, call_sell_strike, put_buy_strike, put_sell_strike,
    entry_call_buy_strike_ask, entry_call_sell_strike_bid, entry_put_buy_strike_ask, entry_put_sell_strike_bid,
    exit_call_buy_strike_bid, exit_call_sell_strike_ask, exit_put_buy_strike_bid, exit_put_sell_strike_ask):
    """
    calculate percentage return for an iron condor spread
     - proceeds will be the initial net credit received less the value of each leg of the spread at close out
    """
    net_credit = (entry_call_sell_strike_bid + entry_put_sell_strike_bid) - (entry_call_buy_strike_ask + entry_put_buy_strike_ask)
    call_strike_diff = call_buy_strike - call_sell_strike
    put_strike_diff = put_sell_strike - put_buy_strike
   
    margin = max(call_strike_diff, put_strike_diff) - net_credit
    
    #handle negative margin anomoly (condition is unlikely to persist on an actual trade but does appear based on closing bid and ask prices)
    #simply set margin to 1 cent to prevent the return from becoming infinite or negative
    if margin <= 0:
        margin = .01
    
    close_out_cost = (exit_call_sell_strike_ask - exit_call_buy_strike_bid) + (exit_put_sell_strike_ask - exit_put_buy_strike_bid)
    pct_return = round((net_credit - close_out_cost) / margin, 3)
    return pct_return

In [7]:
def get_moneyness_bucket(option_moneyness):
    """
    get then moneyness bucket that the option is closest to,
    - buckets range from .05 to .95 in increments of .05
    """
    lower_bucket = round((option_moneyness * 1000 - option_moneyness * 1000 % 50) / 1000, 3)
    upper_bucket = round(lower_bucket + .05, 3)
 
    if option_moneyness - lower_bucket < upper_bucket - option_moneyness:
        return lower_bucket
    else:
        return upper_bucket 
      

In [8]:
def get_nearest_bucket_options(option_type):
    """
    parameters:
    option_type = call | put
    - assign a moneyness bucket to each option record from .10 through to .90 in increments of .05
    - drop .95 bucket as anything above this was dropped during the data capture process
    - drop .05 bucket as these will have fewer responses as a result of some options not getting a meaningful bid (also for symetry reasons)
    - for each moneyness bucket, keep only the strike price with the moneyness closest to the bucket value
    - sort options by moneyness bucket within entry date
    """
    moneyness_column = 'entry_call_moneyness'
    if option_type == 'put':
        moneyness_column = 'entry_put_moneyness'
       
    spy_1wk_options = pd.read_csv('data/spy_1wk_options.csv', parse_dates=['entry_date', 'exit_date', 'expiry_date'])
    spy_1wk_options['moneyness_bucket'] = spy_1wk_options.apply(lambda x: get_moneyness_bucket(x[moneyness_column]), axis=1)  
    spy_1wk_options['bucket_distance'] = abs(spy_1wk_options[moneyness_column] - spy_1wk_options['moneyness_bucket'])
    near_bucket_strikes = spy_1wk_options.groupby(['entry_date', 'moneyness_bucket'])['bucket_distance'].min().to_frame(name='bucket_distance').reset_index()
    sample_strikes = pd.merge(spy_1wk_options, near_bucket_strikes, how='inner', on=['entry_date', 'moneyness_bucket', 'bucket_distance'])
    
    #drop duplicates in case two options were equidistant from the same bucket
    sample_strikes.drop_duplicates(subset=['entry_date', 'moneyness_bucket'], inplace=True)
    
    #drop outer buckets and sort dataframe
    sample_strikes = sample_strikes[(sample_strikes['moneyness_bucket'] > .05) & (sample_strikes['moneyness_bucket'] < .95)]
    sample_strikes = sample_strikes.sort_values(['entry_date', 'moneyness_bucket'], ascending=[True, False]).reset_index(drop=True)
    return sample_strikes
    

#### Strategy: Buy Call
Calculate returns for each Strike Price Bucket

In [57]:
def buy_call():
    """
    Calculate returns for Buy Call Strategy
    """
    sample_strikes = get_nearest_bucket_options('call')
    sample_strikes['pct_return'] = sample_strikes.apply(lambda x: buy_option_return(x['entry_call_ask'], x['exit_call_price']), axis=1)
    
    filepath = 'data/buy_call_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        for row in sample_strikes.itertuples():
                  
            #assign win and loss flags
            win = 1
            loss = 0
            if row.pct_return < 0:
                win = 0
                loss = 1

            record = [
                datetime.datetime.date(row.entry_date),
                round(row.moneyness_bucket * 100, 0),
                row.pct_return,
                win,
                loss,
                row.market_direction,
                row.sampling_key]

            out_csv.writerow(record) 
            
        #upload file to Google Cloud Storage
        upload_blob(filepath)
    
#run function
buy_call()
    

File data/buy_call_returns.csv uploaded to analytics/buy_call_returns.csv.


#### Strategy: Buy Put
Calculate returns for each Strike Price Bucket

In [58]:
def buy_put():
    """
    Calculate returns for Buy Put Strategy
    """
    sample_strikes = get_nearest_bucket_options('put')
    sample_strikes['pct_return'] = sample_strikes.apply(lambda x: buy_option_return(x['entry_put_ask'], x['exit_put_price']), axis=1)
   
    filepath = 'data/buy_put_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        for row in sample_strikes.itertuples():
                
            #assign win and loss flags
            win = 1
            loss = 0
            if row.pct_return < 0:
                win = 0
                loss = 1

            record = [
                datetime.datetime.date(row.entry_date),
                round(row.moneyness_bucket * 100, 0),
                row.pct_return,
                win,
                loss,
                row.market_direction,
                row.sampling_key]

            out_csv.writerow(record) 

        #upload file to Google Cloud Storage
        upload_blob(filepath)
    
#run function
buy_put()

File data/buy_put_returns.csv uploaded to analytics/buy_put_returns.csv.


#### Strategy: Buy Call Spread
Calculate returns across all Strike Bucket combinations

In [59]:
def buy_call_spread():
    """
    Calculate returns for Buy Call Spread Strategy
    """
    sample_strikes = get_nearest_bucket_options('call')
    filepath = 'data/buy_call_spread_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'sell_strike_bucket',
            'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        entry_dates = sample_strikes['entry_date'].unique()
    
        for entry_date in entry_dates:
            option_set = sample_strikes[sample_strikes['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike price record with every other strike price record
            for i in range(row_count):
                for j in range(i + 1, row_count):
 
                    buy_strike_record = option_set.iloc[i]
                    sell_strike_record = option_set.iloc[j]

                    pct_return = buy_spread_return(
                            buy_strike_record['entry_call_ask'],
                            sell_strike_record['entry_call_bid'],
                            buy_strike_record['exit_call_price'],
                            sell_strike_record['exit_call_price'])

                    #assign win and loss flags
                    win = 1
                    loss = 0
                    if pct_return < 0:
                        win = 0
                        loss = 1

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['moneyness_bucket'] * 100, 0),
                        round(sell_strike_record['moneyness_bucket'] * 100, 0),
                        pct_return,
                        win,
                        loss,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)   
                    
        #upload file to Google Cloud Storage
        upload_blob(filepath)

#run function
buy_call_spread()
                    

File data/buy_call_spread_returns.csv uploaded to analytics/buy_call_spread_returns.csv.


#### Strategy: Buy Put Spread
Calculate returns across all Strike Bucket combinations


In [60]:
def buy_put_spread():
    """
    Calculate returns for Buy Put Strategy
    """
    sample_strikes = get_nearest_bucket_options('put')
    filepath = 'data/buy_put_spread_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'sell_strike_bucket',
            'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        entry_dates = sample_strikes['entry_date'].unique()
    
        for entry_date in entry_dates:
            option_set = sample_strikes[sample_strikes['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike price record with every other strike price record
            for i in range(row_count):
                for j in range(i + 1, row_count):

                    buy_strike_record = option_set.iloc[i]
                    sell_strike_record = option_set.iloc[j]

                    pct_return = buy_spread_return(
                            buy_strike_record['entry_put_ask'],
                            sell_strike_record['entry_put_bid'],
                            buy_strike_record['exit_put_price'],
                            sell_strike_record['exit_put_price'])

                    #assign win and loss flags
                    win = 1
                    loss = 0
                    if pct_return < 0:
                        win = 0
                        loss = 1

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['moneyness_bucket'] * 100, 0),
                        round(sell_strike_record['moneyness_bucket'] * 100, 0),
                        pct_return,
                        win,
                        loss,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)   
                    
        #upload file to Google Cloud Storage
        upload_blob(filepath)

#run function
buy_put_spread()

File data/buy_put_spread_returns.csv uploaded to analytics/buy_put_spread_returns.csv.


#### Strategy: Sell Call Spread
Calculate returns across all Strike Bucket combinations

In [61]:
def sell_call_spread():
    """
    Calculate returns for Sell Call Spread Strategy
    """
    sample_strikes = get_nearest_bucket_options('call')
    filepath = 'data/sell_call_spread_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'sell_strike_bucket',
            'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        entry_dates = sample_strikes['entry_date'].unique()
    
        for entry_date in entry_dates:
            option_set = sample_strikes[sample_strikes['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike price record with every other strike price record
            for i in range(row_count):
                for j in range(i + 1, row_count):
 
                    sell_strike_record = option_set.iloc[i]
                    buy_strike_record = option_set.iloc[j]
 
                    pct_return = sell_spread_return(
                            buy_strike_record['strike_price'],
                            sell_strike_record['strike_price'],
                            buy_strike_record['entry_call_ask'],
                            sell_strike_record['entry_call_bid'],
                            buy_strike_record['exit_call_price'],
                            sell_strike_record['exit_call_price'])

                    #assign win and loss flags
                    win = 1
                    loss = 0
                    if pct_return < 0:
                        win = 0
                        loss = 1

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['moneyness_bucket'] * 100, 0),
                        round(sell_strike_record['moneyness_bucket'] * 100, 0),
                        pct_return,
                        win,
                        loss,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)   
                    
        #upload file to Google Cloud Storage
        upload_blob(filepath)

#run function
sell_call_spread()
                    

File data/sell_call_spread_returns.csv uploaded to analytics/sell_call_spread_returns.csv.


#### Strategy: Sell Put Spread
Calculate returns across all Strike Bucket combinations

In [62]:
def sell_put_spread():
    """
    Calculate returns for Sell Put Spread Strategy
    """
    sample_strikes = get_nearest_bucket_options('put')
    filepath = 'data/sell_put_spread_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'sell_strike_bucket',
            'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        entry_dates = sample_strikes['entry_date'].unique()
    
        for entry_date in entry_dates:
            option_set = sample_strikes[sample_strikes['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike price record with every other strike price record
            for i in range(row_count):
                for j in range(i + 1, row_count):
 
                    sell_strike_record = option_set.iloc[i]
                    buy_strike_record = option_set.iloc[j]
 
                    pct_return = sell_spread_return(
                            buy_strike_record['strike_price'],
                            sell_strike_record['strike_price'],
                            buy_strike_record['entry_put_ask'],
                            sell_strike_record['entry_put_bid'],
                            buy_strike_record['exit_put_price'],
                            sell_strike_record['exit_put_price'])

                    #assign win and loss flags
                    win = 1
                    loss = 0
                    if pct_return < 0:
                        win = 0
                        loss = 1

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['moneyness_bucket'] * 100, 0),
                        round(sell_strike_record['moneyness_bucket'] * 100, 0),
                        pct_return,
                        win,
                        loss,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)   
                    
        #upload file to Google Cloud Storage
        upload_blob(filepath)

#run function
sell_put_spread()

File data/sell_put_spread_returns.csv uploaded to analytics/sell_put_spread_returns.csv.


#### Strategy: Sell Iron Condor
This is a neutral strategy so intuitively we want the call spread to lie above the current stock price the put spread to lie below the current stock price.
<br/>
We will start with an Iron Butterfly at the center and work our way to the farthest out of the money call and the farthest out of the money put

<table align='left'>
    <tr>
        <td>Call Sell Strike Bucket</td>
        <td>Call Buy Strike Bucket</td>
        <td>Put Sell Strike Bucket</td>     
        <td>Put Buy Strike Bucket</td>    
    </tr>
    <tr>
        <td>50</td>
        <td>45</td>
        <td>50</td>     
        <td>45</td>    
    </tr>
    <tr>
        <td>50</td>
        <td>40</td>
        <td>50</td>     
        <td>40</td>    
    <tr>
        <td>. . .</td>
        <td>10</td>
        <td>. . .</td>     
        <td>10</td>   
    <tr>
        <td>45</td>
        <td>40</td>
        <td>45</td>     
        <td>40</td>    
    </tr>    
    <tr>
        <td>45</td>
        <td>35</td>
        <td>45</td>     
        <td>35</td>    
    </tr>
        <td>. . .</td>
        <td>10</td>
        <td>. . .</td>     
        <td>10</td>    
    <tr>
        <td>. . .</td>
        <td>. . .</td>
        <td>. . .</td>     
        <td>. . .</td>    
    </tr>
    <tr>
        <td>20</td>
        <td>10</td>
        <td>20</td>     
        <td>10</td>    
    </tr>
</table>


In [25]:
def sell_iron_condor():
    """
    Calculate returns for Sell Iron Condor Strategy
    """
    #get call options
    call_sample_strikes = get_nearest_bucket_options('call')
    call_sample_strikes = call_sample_strikes[call_sample_strikes['moneyness_bucket'] <= .5]
    
    call_sample_strikes = call_sample_strikes[['entry_date', 'strike_price', 'moneyness_bucket', 
        'entry_call_bid', 'entry_call_ask', 'exit_call_price', 'entry_stock_price', 'exit_stock_price', 'market_direction', 'sampling_key']]
    
    call_sample_strikes.rename(columns={'strike_price': 'call_strike_price'}, inplace=True)
    
    #get put options
    put_sample_strikes = get_nearest_bucket_options('put')
    put_sample_strikes = put_sample_strikes[put_sample_strikes['moneyness_bucket'] <= .5]
    put_sample_strikes = put_sample_strikes[['entry_date', 'strike_price', 'moneyness_bucket', 'entry_put_bid', 'entry_put_ask', 'exit_put_price']]
    put_sample_strikes.rename(columns={'strike_price': 'put_strike_price'}, inplace=True)
    
    #combine call and put options
    #call and put options having the same moneyness bucket will be out of the money by the same amount,
    #but on opposite sites of the current stock price
    sample_strikes = pd.merge(call_sample_strikes, put_sample_strikes, how='inner', on=['entry_date', 'moneyness_bucket'])    
    sample_strikes = sample_strikes.sort_values(['entry_date', 'moneyness_bucket'], ascending=[True, False]).reset_index(drop=True)
        
    filepath = 'data/sell_iron_condor_returns.csv'
    
    with open(filepath, 'w', newline='') as f:
        out_csv = csv.writer(f)
        columns = ['entry_date', 'buy_strike_bucket', 'sell_strike_bucket', 
            'pct_return', 'win', 'loss', 'market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
        
        entry_dates = sample_strikes['entry_date'].unique()
    
        for entry_date in entry_dates:
            option_set = sample_strikes[sample_strikes['entry_date'] == entry_date]
            row_count = option_set.shape[0]

            #pair each strike bucket record with every other strike bucket record
            for i in range(row_count):
                for j in range(i + 1, row_count):
 
                    sell_strike_record = option_set.iloc[i]
                    buy_strike_record = option_set.iloc[j]
                 
                    pct_return = iron_condor_return(
                        buy_strike_record['call_strike_price'], sell_strike_record['call_strike_price'],
                        buy_strike_record['put_strike_price'], sell_strike_record['put_strike_price'],
                        buy_strike_record['entry_call_ask'], sell_strike_record['entry_call_bid'],
                        buy_strike_record['entry_put_ask'], sell_strike_record['entry_put_bid'],
                        buy_strike_record['exit_call_price'], sell_strike_record['exit_call_price'],
                        buy_strike_record['exit_put_price'], sell_strike_record['exit_put_price'])         

                    #assign win and loss flags
                    win = 1
                    loss = 0
                    if pct_return < 0:
                        win = 0
                        loss = 1

                    record = [
                        datetime.datetime.date(buy_strike_record['entry_date']),
                        round(buy_strike_record['moneyness_bucket'] * 100, 0),
                        round(sell_strike_record['moneyness_bucket'] * 100, 0),
                        pct_return,
                        win,
                        loss,
                        buy_strike_record['market_direction'],
                        buy_strike_record['sampling_key']]
                    out_csv.writerow(record)   
                    
        #upload file to Google Cloud Storage
        upload_blob(filepath)

#run function
sell_iron_condor()

File data/sell_iron_condor_returns.csv uploaded to analytics/sell_iron_condor_returns.csv.


In [27]:
test1 = pd.read_csv('data/sell_iron_condor_returns.csv')
idx = test1['pct_return'].idxmax()
test1.iloc[idx]

entry_date              2016-12-09
buy_strike_bucket               40
sell_strike_bucket              50
pct_return                    26.5
win                              1
loss                             0
market_direction      Neutral Down
sampling_key                 0.257
Name: 1974, dtype: object